In [20]:
import numpy as np
import seaborn as sns
import librosa
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
import tensorflow as tf
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
import keras
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.6.0


In [21]:
path_dir = 'data/'

In [22]:
def read_sound(file_path):
    y, sr = librosa.load(file_path, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    return librosa.feature.melspectrogram(y=y, sr=sr)
read_sound('data/neg/AnyConv.com__neg-0421-083-cough-m-53-0.wav')

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
listFiles = os.listdir('data/neg/')
print(listFiles[:5])

['AnyConv.com__neg-0421-083-cough-m-53-0.wav', 'AnyConv.com__neg-0421-083-cough-m-53-1.wav', 'AnyConv.com__neg-0421-083-cough-m-53-11.wav', 'AnyConv.com__neg-0421-083-cough-m-53-12.wav', 'AnyConv.com__neg-0421-083-cough-m-53-13.wav']


In [24]:
#load Benign
listCancer = []
listResults = []
cnt=0
for elem in listFiles:
    img = read_sound('data/neg/'+elem)
    listCancer += [img]
    listResults += [np.array([1])]
       
print(listCancer[:3])
print(img.shape)

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[1.5383340e-04, 5.0781910e-05, 1.4765496e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.8477437e-04, 6.5516302e-05, 1.4249467e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.9289967e-05, 1.6717848e-05, 8.2170897e-07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [1.2430817e-08, 3.2063012e-09, 8.9009154e-11, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.2183010e-08, 3.

In [25]:
import random
listFiles = os.listdir('data/pos/')
print(listFiles[:5])
for elem in listFiles:
    img = read_sound('data/pos/'+elem)
    pz = random.randint(0, len(listCancer))
    listCancer.insert(pz, img)
    listResults.insert(pz, np.array([0]))
print(listResults)

['AnyConv.com__pos-0421-084-cough-m-50-1.wav', 'AnyConv.com__pos-0421-084-cough-m-50-2.wav', 'AnyConv.com__pos-0421-084-cough-m-50-3.wav', 'AnyConv.com__pos-0421-084-cough-m-50-5.wav', 'AnyConv.com__pos-0421-084-cough-m-50-6.wav']
[array([0]), array([1]), array([1]), array([1]), array([0]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([1]), array([0]), array([1]), array([0]), array([0]), array([0]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([1]), array([1]), array([1]), array([1]), array([0]), array([1]), array([1]), array([0]), array([1]), array([1]), array([1]), array([1]), array([0]), array([1]), array([1]), array([0]), array([0]), array([1]), array([1]), array([1]), array([0]), array([1]), array([1]), array([0]), array([1]), array([1]), array([0]), array([1]), array([0]), array([1]), array([1]), array([0]), array([0]), array([1]), array([0]), array([0]), array([1]), array([0]), 

In [26]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x200 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (1,1), activation='relu', input_shape=(2, 64, 95)),
    tf.keras.layers.MaxPooling2D(1, 1),
    # The second convolution
    tf.keras.layers.Conv2D(32, (1,1), activation='relu'),
    tf.keras.layers.MaxPooling2D(1,1),
    # The third convolution
    tf.keras.layers.Conv2D(64, (1,1), activation='relu'),
    tf.keras.layers.MaxPooling2D(1,1),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (1, 1), activation='relu'),
    tf.keras.layers.MaxPooling2D(1, 1),
    # # The fifth convolution
    tf.keras.layers.Conv2D(64, (1, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('dandelions') and 1 for the other ('grass')
    tf.keras.layers.Dense(1, activation='sigmoid')
])
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
opt = Adam(learning_rate = lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 2, 64, 16)         1536      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 2, 64, 16)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 64, 32)         544       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 2, 64, 32)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 2, 64, 64)         2112      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 2, 64, 64)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 2, 64, 64)        

In [27]:
from imblearn.over_sampling import SMOTE
print(len(listCancer))
print(len(listResults))
oversample = SMOTE()
X, y = oversample.fit_resample(np.array(listCancer).reshape(len(listCancer),-1), listResults)
print(len(X))
print(len(y))
listCancer = list(X)
listResults = list(y)
print(listResults)

97
97
108
108
[0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [100]:

def main(pz):
    model = tf.keras.models.Sequential([
        # Note the input shape is the desired size of the image 200x200 with 3 bytes color
        # This is the first convolution
        tf.keras.layers.Conv2D(16, (1,1), activation='relu', input_shape=(2, 64, 95)),
        tf.keras.layers.MaxPooling2D(1, 1),
        # The second convolution
        tf.keras.layers.Conv2D(32, (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D(1,1),
        # The third convolution
        tf.keras.layers.Conv2D(32, (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D(1,1),
        # The fourth convolution
        tf.keras.layers.Conv2D(16, (1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D(1, 1),
        # # The fifth convolution
        tf.keras.layers.Conv2D(8, (1, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(16, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('dandelions') and 1 for the other ('grass')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=0.2)
    opt = Adam(learning_rate = lr_schedule)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

    per = np.random.permutation(len(listCancer))
    ln = int(len(listCancer) * 0.55)
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    val = 1
    while ln > 0:
        val = 1-val
        ln-=1
        for i in range(len(per)):
            if per[i] in positions:
                continue
            if listResults[per[i]] == val:
                positions += [per[i]]
                break
    ln = (len(listCancer) - int(len(listCancer) * 0.55)) // 2
    positions_valid = []
    val = 1
    while ln > 0:
        val = 1-val
        ln-=1
        for i in range(len(per)):
            if per[i] in positions:
                continue
            if per[i] in positions_valid:
                continue
            if listResults[per[i]] == val:
                positions_valid += [per[i]]
                break
    in_test = 1
    x_valid = []
    y_valid = []
    for i in range(len(listCancer)):
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            in_test = 1 - in_test
            if i not in positions_valid:
                x_test += [listCancer[i]]
                y_test += [listResults[i]]
            else:
                x_valid += [listCancer[i]]
                y_valid += [listResults[i]]
    print(len(x_train))
    
    #x_valid = x_test
    #y_valid = y_test
    
    #x_test = x_test[:ln]
    #y_test = y_test[:ln]
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 2, 64, 95)
    y_train = np.array(y_train)
    

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 2, 64, 95)
    y_test = np.array(y_test)
    
    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 2, 64, 95)
    y_valid = np.array(y_valid)
    
    model.fit(x_train, y_train,
                epochs = 100,
                batch_size=16,
                validation_data =(x_valid, y_valid),
                shuffle = True)
    scores = model.evaluate(x_test, y_test, verbose = 1)
    print(model.predict(x_test))
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    ret = model.predict(x_test)
    result = []
    pred = []
    for a,b in zip(ret, y_test):
        if a[0] < 0.5:
            pred+=[0]
        else:
            pred+=[1]
        result+=[b]
    print(pred)
    print(result)
    precision = precision_score(result, pred)
    recall = recall_score(result, pred)
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 Score: ', fmeasure)
    #precisions += [precision]
    #recalls += [recall]
    #model.save('saved_models/Sequential'+pz+'.h5')
    return [scores[1], precision, recall, fmeasure]

In [101]:
accs=[]
precisions = []
recalls = []
fmeasures = []
for i in range(10):
    ret = main(i)
    accs+=[ret[0]]
    precisions+=[ret[1]]
    recalls+=[ret[2]]
    fmeasures+=[ret[3]]
print(accs)
print(precisions)
print(recalls)
print(fmeasures)

59
Epoch 1/100
4/4 [==============================] - 1s 67ms/step - loss: 0.2216 - accuracy: 0.5424 - val_loss: 0.2516 - val_accuracy: 0.6667
Epoch 2/100
4/4 [==============================] - 0s 18ms/step - loss: 0.2081 - accuracy: 0.6780 - val_loss: 0.2531 - val_accuracy: 0.7083
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 0.1915 - accuracy: 0.6949 - val_loss: 0.2547 - val_accuracy: 0.7083
Epoch 4/100
4/4 [==============================] - 0s 21ms/step - loss: 0.1927 - accuracy: 0.6610 - val_loss: 0.2618 - val_accuracy: 0.7083
Epoch 5/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1889 - accuracy: 0.6949 - val_loss: 0.2845 - val_accuracy: 0.6667
Epoch 6/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1835 - accuracy: 0.6949 - val_loss: 0.2846 - val_accuracy: 0.7083
Epoch 7/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1789 - accuracy: 0.7119 - val_loss: 0.2857 - val_accuracy: 0.7083
Epoch 8/100
4/4 [

4/4 [==============================] - 0s 18ms/step - loss: 0.1602 - accuracy: 0.7966 - val_loss: 0.2657 - val_accuracy: 0.7083
Epoch 59/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1602 - accuracy: 0.7966 - val_loss: 0.2702 - val_accuracy: 0.7083
Epoch 60/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1603 - accuracy: 0.7966 - val_loss: 0.2728 - val_accuracy: 0.7083
Epoch 61/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1606 - accuracy: 0.7966 - val_loss: 0.2785 - val_accuracy: 0.7083
Epoch 62/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1607 - accuracy: 0.7966 - val_loss: 0.2780 - val_accuracy: 0.7083
Epoch 63/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1434 - accuracy: 0.8136 - val_loss: 0.2755 - val_accuracy: 0.7083
Epoch 64/100
4/4 [==============================] - 0s 19ms/step - loss: 0.1430 - accuracy: 0.8136 - val_loss: 0.2787 - val_accuracy: 0.7083
Epoch 65/100
4/4 [========

4/4 [==============================] - 0s 17ms/step - loss: 0.1722 - accuracy: 0.7627 - val_loss: 0.3142 - val_accuracy: 0.5417
Epoch 11/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1630 - accuracy: 0.7627 - val_loss: 0.3174 - val_accuracy: 0.5833
Epoch 12/100
4/4 [==============================] - 0s 16ms/step - loss: 0.1634 - accuracy: 0.7966 - val_loss: 0.3389 - val_accuracy: 0.5417
Epoch 13/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1636 - accuracy: 0.7797 - val_loss: 0.3518 - val_accuracy: 0.5000
Epoch 14/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1613 - accuracy: 0.7797 - val_loss: 0.3473 - val_accuracy: 0.5417
Epoch 15/100
4/4 [==============================] - 0s 16ms/step - loss: 0.1551 - accuracy: 0.7966 - val_loss: 0.3375 - val_accuracy: 0.5833
Epoch 16/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1531 - accuracy: 0.7966 - val_loss: 0.3319 - val_accuracy: 0.5833
Epoch 17/100
4/4 [========

4/4 [==============================] - 0s 16ms/step - loss: 0.1163 - accuracy: 0.8475 - val_loss: 0.3587 - val_accuracy: 0.5833
Epoch 69/100
4/4 [==============================] - 0s 16ms/step - loss: 0.1163 - accuracy: 0.8475 - val_loss: 0.3588 - val_accuracy: 0.5833
Epoch 70/100
4/4 [==============================] - 0s 19ms/step - loss: 0.1166 - accuracy: 0.8475 - val_loss: 0.3584 - val_accuracy: 0.5833
Epoch 71/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1172 - accuracy: 0.8475 - val_loss: 0.3602 - val_accuracy: 0.5833
Epoch 72/100
4/4 [==============================] - 0s 16ms/step - loss: 0.1164 - accuracy: 0.8475 - val_loss: 0.3598 - val_accuracy: 0.5833
Epoch 73/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1160 - accuracy: 0.8475 - val_loss: 0.3584 - val_accuracy: 0.5833
Epoch 74/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1163 - accuracy: 0.8475 - val_loss: 0.3577 - val_accuracy: 0.5833
Epoch 75/100
4/4 [========

4/4 [==============================] - 0s 16ms/step - loss: 0.1913 - accuracy: 0.7627 - val_loss: 0.3022 - val_accuracy: 0.6250
Epoch 22/100
4/4 [==============================] - 0s 17ms/step - loss: 0.2006 - accuracy: 0.7627 - val_loss: 0.3057 - val_accuracy: 0.6250
Epoch 23/100
4/4 [==============================] - 0s 17ms/step - loss: 0.2613 - accuracy: 0.6949 - val_loss: 0.2878 - val_accuracy: 0.6250
Epoch 24/100
4/4 [==============================] - 0s 17ms/step - loss: 0.2149 - accuracy: 0.7458 - val_loss: 0.2535 - val_accuracy: 0.6667
Epoch 25/100
4/4 [==============================] - 0s 16ms/step - loss: 0.2050 - accuracy: 0.7627 - val_loss: 0.2529 - val_accuracy: 0.6667
Epoch 26/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1605 - accuracy: 0.7966 - val_loss: 0.2494 - val_accuracy: 0.6667
Epoch 27/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1551 - accuracy: 0.8136 - val_loss: 0.2509 - val_accuracy: 0.6667
Epoch 28/100
4/4 [========

...

4/4 [==============================] - 0s 17ms/step - loss: 0.1352 - accuracy: 0.8305 - val_loss: 0.2700 - val_accuracy: 0.6667
Epoch 73/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1343 - accuracy: 0.8305 - val_loss: 0.2669 - val_accuracy: 0.6667
Epoch 74/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1339 - accuracy: 0.8305 - val_loss: 0.2652 - val_accuracy: 0.6667
Epoch 75/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1330 - accuracy: 0.8305 - val_loss: 0.2669 - val_accuracy: 0.6667
Epoch 76/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1319 - accuracy: 0.8305 - val_loss: 0.2675 - val_accuracy: 0.6667
Epoch 77/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1308 - accuracy: 0.8305 - val_loss: 0.2668 - val_accuracy: 0.6667
Epoch 78/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1300 - accuracy: 0.8305 - val_loss: 0.2275 - val_accuracy: 0.7083
Epoch 79/100
4/4 [========

4/4 [==============================] - 0s 16ms/step - loss: 0.1739 - accuracy: 0.7966 - val_loss: 0.2157 - val_accuracy: 0.7917
Epoch 25/100
4/4 [==============================] - 0s 15ms/step - loss: 0.2598 - accuracy: 0.6610 - val_loss: 0.2275 - val_accuracy: 0.7917
Epoch 26/100
4/4 [==============================] - 0s 16ms/step - loss: 0.2629 - accuracy: 0.6610 - val_loss: 0.2571 - val_accuracy: 0.7500
Epoch 27/100
4/4 [==============================] - 0s 15ms/step - loss: 0.2611 - accuracy: 0.6780 - val_loss: 0.2507 - val_accuracy: 0.7500
Epoch 28/100
4/4 [==============================] - ETA: 0s - loss: 0.3408 - accuracy: 0.62 - 0s 16ms/step - loss: 0.2599 - accuracy: 0.6780 - val_loss: 0.2362 - val_accuracy: 0.7917
Epoch 29/100
4/4 [==============================] - 0s 15ms/step - loss: 0.2548 - accuracy: 0.6610 - val_loss: 0.2353 - val_accuracy: 0.7917
Epoch 30/100
4/4 [==============================] - 0s 16ms/step - loss: 0.2536 - accuracy: 0.6780 - val_loss: 0.2324 - val_a

Epoch 82/100
4/4 [==============================] - 0s 17ms/step - loss: 0.2482 - accuracy: 0.6780 - val_loss: 0.1867 - val_accuracy: 0.8333
Epoch 83/100
4/4 [==============================] - 0s 16ms/step - loss: 0.2481 - accuracy: 0.6780 - val_loss: 0.1870 - val_accuracy: 0.8333
Epoch 84/100
4/4 [==============================] - 0s 16ms/step - loss: 0.2482 - accuracy: 0.6780 - val_loss: 0.1867 - val_accuracy: 0.8333
Epoch 85/100
4/4 [==============================] - 0s 17ms/step - loss: 0.2481 - accuracy: 0.6780 - val_loss: 0.1866 - val_accuracy: 0.8333
Epoch 86/100
4/4 [==============================] - 0s 16ms/step - loss: 0.2481 - accuracy: 0.6780 - val_loss: 0.1863 - val_accuracy: 0.8333
Epoch 87/100
4/4 [==============================] - 0s 18ms/step - loss: 0.2482 - accuracy: 0.6780 - val_loss: 0.1858 - val_accuracy: 0.8333
Epoch 88/100
4/4 [==============================] - 0s 17ms/step - loss: 0.2481 - accuracy: 0.6780 - val_loss: 0.1858 - val_accuracy: 0.8333
Epoch 89/100


4/4 [==============================] - 0s 17ms/step - loss: 0.1578 - accuracy: 0.8136 - val_loss: 0.2751 - val_accuracy: 0.6250
Epoch 35/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1588 - accuracy: 0.8136 - val_loss: 0.2814 - val_accuracy: 0.6250
Epoch 36/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1577 - accuracy: 0.8136 - val_loss: 0.2814 - val_accuracy: 0.6250
Epoch 37/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1567 - accuracy: 0.8136 - val_loss: 0.2778 - val_accuracy: 0.6250
Epoch 38/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1569 - accuracy: 0.8136 - val_loss: 0.2715 - val_accuracy: 0.6250
Epoch 39/100
4/4 [==============================] - 0s 16ms/step - loss: 0.1571 - accuracy: 0.8136 - val_loss: 0.2678 - val_accuracy: 0.6250
Epoch 40/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1570 - accuracy: 0.8136 - val_loss: 0.2672 - val_accuracy: 0.6250
Epoch 41/100
4/4 [========

4/4 [==============================] - 0s 18ms/step - loss: 0.1362 - accuracy: 0.8305 - val_loss: 0.2520 - val_accuracy: 0.6667
Epoch 93/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1363 - accuracy: 0.8305 - val_loss: 0.2587 - val_accuracy: 0.6667
Epoch 94/100
4/4 [==============================] - 0s 16ms/step - loss: 0.1347 - accuracy: 0.8305 - val_loss: 0.2615 - val_accuracy: 0.6667
Epoch 95/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1345 - accuracy: 0.8305 - val_loss: 0.3042 - val_accuracy: 0.6250
Epoch 96/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1340 - accuracy: 0.8305 - val_loss: 0.2993 - val_accuracy: 0.6250
Epoch 97/100
4/4 [==============================] - 0s 16ms/step - loss: 0.1360 - accuracy: 0.8305 - val_loss: 0.2490 - val_accuracy: 0.6667
Epoch 98/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1363 - accuracy: 0.8305 - val_loss: 0.2443 - val_accuracy: 0.6667
Epoch 99/100
4/4 [========

In [102]:
print(len(accs))
print('accs: ', accs)
print('precisions: ', precisions)
print('recalls: ', recalls)
print('fmeasures: ', fmeasures)

10
accs:  [0.6000000238418579, 0.800000011920929, 0.7200000286102295, 0.6800000071525574, 0.800000011920929, 0.6800000071525574, 0.6399999856948853, 0.7599999904632568, 0.7200000286102295, 0.7200000286102295]
precisions:  [0.6153846153846154, 0.75, 0.65, 0.6470588235294118, 0.7222222222222222, 0.6923076923076923, 0.625, 0.7058823529411765, 0.6875, 0.75]
recalls:  [0.6153846153846154, 0.9230769230769231, 1.0, 0.8461538461538461, 1.0, 0.6923076923076923, 0.7692307692307693, 0.9230769230769231, 0.8461538461538461, 0.6923076923076923]
fmeasures:  [0.6153846153846154, 0.8275862068965517, 0.787878787878788, 0.7333333333333334, 0.8387096774193548, 0.6923076923076923, 0.6896551724137931, 0.8000000000000002, 0.7586206896551724, 0.7199999999999999]


In [103]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.6656641217926377, 0.7583359030028946)

In [109]:
from sklearn.linear_model import LogisticRegression

accs=[]
precisions=[]
recalls=[]
fmeasures=[]

for tst in range(10):
    x_train=[]
    y_train=[]
    model  = LogisticRegression()
    per = np.random.permutation(len(listCancer))
    ln = int(len(listCancer) * 0.8)
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    val = 1
    while ln > 0:
        val = 1-val
        ln-=1
        for i in range(len(per)):
            if per[i] in positions:
                continue
            if listResults[per[i]] == val:
                positions += [per[i]]
                break
    for i in range(len(listCancer)):
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            x_test += [listCancer[i]]
            y_test += [listResults[i]]
    print(len(x_train))

    x_valid = x_test
    y_valid = y_test

    #x_test = x_test[:ln]
    #y_test = y_test[:ln]

    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 128* 95)
    y_train = np.array(y_train)


    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 128* 95)
    y_test = np.array(y_test)

    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 128* 95)
    y_valid = np.array(y_valid)

    model.fit(x_train, y_train)

    print(model.predict(x_test))
    print(y_test)
    pred = model.predict(x_test)
    print(pred)
    print(y_train)
    result=[]
    nr=0
    for i in range(len(pred)):
        val = y_test[i]
        if pred[i] == val:
            nr+=1
        result+=[val]
    acc = nr / len(pred)
    precision = precision_score(result, pred)
    recall = recall_score(result, pred)
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred)
    accs+=[acc]
    precisions+=[precision]
    recalls+=[recall]
    fmeasures+=[fmeasure]

print("accs: ", accs)
print("precisions: ", precisions)
print("recalls: ", recalls)
print("fmeasures: ", fmeasures)

86
[1 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0]
[0 0 0 0 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 0 0]
[1 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0]
[1 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0
 0 1 1 0 0 0 0 0 0 0 0 0]
86
[0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 1]
[0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0]
[0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 1]
[0 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 0 0 1
 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0
 1 1 0 0 1 1 0 0 0 0 0 0]
86
[1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0]
[1 1 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0]
[1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0]
[0 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 1
 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 0
 0 1 1 0 0 0 0 0 0 0 0 0]
86
[1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 1 0 0]


In [185]:
import pickle

#
# Create your model here (same as above)
#

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [1]:
accs = [0.6136363744735718, 0.5909090638160706, 0.5454545617103577, 0.6818181872367859, 0.6818181872367859, 0.7045454382896423, 0.5681818127632141, 0.6136363744735718, 0.5681818127632141, 0.6818181872367859]
precisions = [0.5945945945945946, 0.5945945945945946, 0.5, 0.675, 0.6666666666666666, 0.7407407407407407, 0.5681818181818182, 0.6071428571428571, 0.5365853658536586, 0.6923076923076923]
recalls = [0.9166666666666666, 0.88, 0.85, 0.9642857142857143, 0.88, 0.7692307692307693, 1.0, 0.7391304347826086, 1.0, 0.75]
fmeasures = [0.7213114754098361, 0.7096774193548386, 0.6296296296296295, 0.7941176470588236, 0.7586206896551725, 0.7547169811320754, 0.7246376811594203, 0.6666666666666666, 0.6984126984126985, 0.7199999999999999]

In [104]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.6656641217926377, 0.7583359030028946)

In [105]:
#define sample data
data = precisions

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.649620505046403, 0.7194506362306208)

In [106]:
#define sample data
data = recalls

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.7344056991297674, 0.9271327624086942)

In [107]:
#define sample data
data = fmeasures

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.6963102746761899, 0.7963849603816703)

In [17]:
#pt logistic de acuma

In [18]:
accs=  [0.7, 0.9, 0.6, 0.65, 0.45, 0.6, 0.7, 0.7, 0.65, 0.75]
precisions=  [0.7142857142857143, 0.8571428571428571, 0.6153846153846154, 0.6111111111111112, 0.35714285714285715, 0.625, 0.6666666666666666, 0.7142857142857143, 0.6428571428571429, 0.7058823529411765]
recalls=  [0.8333333333333334, 1.0, 0.7272727272727273, 1.0, 0.7142857142857143, 0.8333333333333334, 0.9090909090909091, 0.8333333333333334, 0.8181818181818182, 1.0]
fmeasures=  [0.7692307692307692, 0.923076923076923, 0.6666666666666667, 0.7586206896551725, 0.4761904761904762, 0.7142857142857143, 0.7692307692307692, 0.7692307692307692, 0.7200000000000001, 0.8275862068965517]

In [110]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.6863553741647672, 0.731826444017051)

In [111]:
#define sample data
data = precisions

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.643779912347533, 0.7020409334733131)

In [112]:
#define sample data
data = recalls

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.7703306403948624, 0.8842148141505919)

In [113]:
#define sample data
data = fmeasures

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.717634894932947, 0.7605908632058547)